In [ ]:
import logging
import os
import asyncio

from ogmyrag.report_retrieval.report_retrieval import ReportRetrievalManager
from ogmyrag.report_retrieval.retrieval_storage import RetrievalAsyncStorageManager
from ogmyrag.report_scraper.models import ReportType
from ogmyrag.my_logging import configure_logger
from ogmyrag.storage import PineconeStorage
from ogmyrag.report_retrieval.report_chunker import rag_answer_with_company_detection

from dotenv import load_dotenv

retrieval_logger = configure_logger(name='query',log_level=logging.INFO, log_file='logs/query.log')
retrieval_logger.info("\n" + "=" * 80)

load_dotenv(override=True)

mongo_db_uri = os.getenv("MONGO_DB_URI_JJ","")
pinecone_api_key = os.getenv("PINECONE_API_KEY_JJ","")
genai_api_key = os.getenv("GENAI_API_KEY_JJ","")
openai_api_key = os.getenv("OPENAI_API_KEY", "")

INDEX_NAME = "company-disclosures-index"
EMBED_MODEL = "text-embedding-3-small"
DIMENSION = 1536
GENAI_MODEL = "gemini-2.5-pro"
#GENAI_MODEL = "gemini-2.5-flash"
OPENAI_MODEL = "gpt-5-nano"

db_name = "FYP"
storage = RetrievalAsyncStorageManager(mongo_uri=mongo_db_uri, db_name=db_name)


pine = PineconeStorage(
    pinecone_api_key = pinecone_api_key,
    openai_api_key = openai_api_key
)

pine.create_index_if_not_exists(
    index_name=entity_vector_config["index_name"],
    dimension=entity_vector_config["pinecone_dimensions"],
    metric=entity_vector_config["pinecone_metric"],
    cloud=entity_vector_config["pinecone_cloud"],
    region=entity_vector_config["pinecone_environment"],
)

pine = PineconeStorage(
    index_name = INDEX_NAME,
    pinecone_api_key = pinecone_api_key,
    pinecone_environment = "us-east-1",
    pinecone_cloud = "aws",
    pinecone_metric = "cosine",
    pinecone_dimensions = DIMENSION,
    openai_api_key = openai_api_key
)

2025-08-28 13:23:39,515 - query - INFO - 


## Simple User Query

In [2]:
out = await rag_answer_with_company_detection(
    pine,
    query = "What is revenue for this company?",
    top_k = 5
)

2025-08-28 13:23:47,217 - query - INFO - RAG start | query='What is revenue for this company?' | top_k=5
2025-08-28 13:23:49,786 - query - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-28 13:23:55,047 - query - INFO - Detected company: None
2025-08-28 13:23:55,048 - query - INFO - Search query unchanged.
2025-08-28 13:23:55,048 - query - INFO - Query filter: {}
2025-08-28 13:24:13,718 - query - INFO - Final answer: 

VETECE_HOLDINGS_BERHAD
- Latest revenue (FPE 2024): RM 11,265,000 (Total revenue)

EDELTEQ_HOLDINGS_BERHAD
- Not found in provided documents.

ICT_ZONE_ASIA_BERHAD
- Revenue (FYE 2024, Company): RM 114,428,000 (Total revenue)

AUTOCOUNT_DOTCOM_BERHAD
- Revenue (Company 2024): RM 11,010,000


2025-08-28 13:24:13,720 - query - INFO - Token usage | total = 7466


In [5]:
out = await rag_answer_with_company_detection(
    pine,
    query = "What is farm fresh management’s strategy for growth over the next 3–5 years?",
    top_k = 5
)

2025-08-27 11:57:27,006 - retrieval - INFO - RAG start | query='What is farm fresh management’s strategy for growth over the next 3–5 years?' | top_k=5
2025-08-27 11:57:27,570 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 11:57:44,240 - retrieval - INFO - Detected company: 'FARM_FRESH_BERHAD'
2025-08-27 11:57:44,243 - retrieval - INFO - Search query normalized: 'What is farm fresh management’s strategy for growth over the next 3–5 years?' → "What is the company's strategy for growth over the next 3–5 years?"
2025-08-27 11:57:44,243 - retrieval - INFO - Query filter: {'from_company': 'FARM_FRESH_BERHAD'}
2025-08-27 11:57:58,769 - retrieval - INFO - Final answer: 

- Product portfolio expansion: Focus on developing and diversifying products (CPG ice cream via the Inside Scoop brand by Aug 2024; expanding into plant-based yoghurt;

In [2]:
out = await rag_answer_with_company_detection(
    pine,
    query = "Who is Joyce Wong Ai May? What is his/her position? Elaborate more",
    top_k = 5
)

2025-08-27 12:07:42,060 - retrieval - INFO - RAG start | query='Who is Joyce Wong Ai May? What is his/her position? Elaborate more' | top_k=5
2025-08-27 12:07:44,310 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:07:48,327 - retrieval - INFO - Detected company: None
2025-08-27 12:07:48,328 - retrieval - INFO - Search query unchanged.
2025-08-27 12:07:48,329 - retrieval - INFO - Query filter: {}
2025-08-27 12:08:02,078 - retrieval - INFO - Final answer: 

- Joyce Wong Ai May is an Independent Non-Executive Director of Edelteq Holdings Berhad; appointed to the Board on 1 August 2022. She is Malaysian, aged 47.

- Education and professional background:
  - Bachelor of Commerce, University of Tasmania (Dec 1998)
  - Accountant by profession; CPA Australia since Jan 2003
  - Member of the Malaysian Institute of Accountants since S

In [3]:
out = await rag_answer_with_company_detection(
    pine,
    query = "Who is Liew Soung Yue? What is his/her position? Elaborate more",
    top_k = 5
)

2025-08-27 12:08:21,470 - retrieval - INFO - RAG start | query='Who is Liew Soung Yue? What is his/her position? Elaborate more' | top_k=5
2025-08-27 12:08:22,282 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:08:27,041 - retrieval - INFO - Detected company: None
2025-08-27 12:08:27,042 - retrieval - INFO - Search query unchanged.
2025-08-27 12:08:27,042 - retrieval - INFO - Query filter: {}
2025-08-27 12:08:33,375 - retrieval - INFO - Final answer: 

- Liew Soung Yue is an Independent Non-Executive Director of AutoCount DotCom Berhad.
- Board appointment: Appointed to the Board on 6 April 2022.
- Committee roles: Chairman of the Nomination Committee; member of the Audit and Risk Management Committee and the Remuneration Committee.
- Education: Doctor of Philosophy (Information Engineering) from The Chinese University of Hong

In [4]:
out = await rag_answer_with_company_detection(
    pine,
    query = "Who are the board of directors of AutoCount? Tell more story or background about each of them.",
    top_k = 5
)

2025-08-27 12:08:46,790 - retrieval - INFO - RAG start | query='Who are the board of directors of AutoCount? Tell more story or background about each of them.' | top_k=5
2025-08-27 12:08:47,632 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:08:55,147 - retrieval - INFO - Detected company: 'AUTOCOUNT_DOTCOM_BERHAD'
2025-08-27 12:08:55,148 - retrieval - INFO - Search query normalized: 'Who are the board of directors of AutoCount? Tell more story or background about each of them.' → 'Who are the board of directors of the company? Tell more story or background about each of them.'
2025-08-27 12:08:55,149 - retrieval - INFO - Query filter: {'from_company': 'AUTOCOUNT_DOTCOM_BERHAD'}
2025-08-27 12:09:12,354 - retrieval - INFO - Final answer: 

AUTOCOUNT_DOTCOM_BERHAD — Board of Directors (Directors)

- Choo Chin Peng (CCP) — Execut

In [4]:
out = await rag_answer_with_company_detection(
    pine,
    query = "What are the mission and vision of EDELTEQ?",
    top_k = 5
)

2025-08-27 19:42:23,422 - query - INFO - RAG start | query='What are the mission and vision of EDELTEQ?' | top_k=5
2025-08-27 19:42:25,816 - query - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 19:42:32,845 - query - INFO - Detected company: 'EDELTEQ_HOLDINGS_BERHAD'
2025-08-27 19:42:32,846 - query - INFO - Search query normalized: 'What are the mission and vision of EDELTEQ?' → 'What are the mission and vision of the company?'
2025-08-27 19:42:32,846 - query - INFO - Query filter: {'from_company': 'EDELTEQ_HOLDINGS_BERHAD'}
2025-08-27 19:42:41,156 - query - INFO - Final answer: 

- Vision: "to convert all our customers’ manufacturing ideas into reality; and enhance their manufacturing process & system efficiency."
- Mission: "to bring Value propositions, Innovative solutions and Niche processes to support semiconductor manufacturing needs 

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    query = "What the company CABNET do?",
    top_k = 5
)

2025-08-26 12:05:49,090 - retrieval - INFO - RAG start | query='What the company CABNET do?' | top_k=5
2025-08-26 12:05:49,670 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-26 12:06:00,430 - retrieval - INFO - Detected company: 'CABNET_HOLDINGS_BERHAD'
2025-08-26 12:06:00,431 - retrieval - INFO - Search query normalized: 'What the company CABNET do?' → 'What the company do?'
2025-08-26 12:06:00,432 - retrieval - INFO - Query filter: {'from_company': 'CABNET_HOLDINGS_BERHAD'}
2025-08-26 12:06:05,128 - retrieval - INFO - Final answer: 

Cabnet Holdings Berhad provides building management solutions, including structured cabling works and ELV systems for buildings and facilities, and IT services. Their activities cover design, supply, build, testing and commissioning, plus project management, training, maintenance and aftersales servi

In [7]:
out = await rag_answer_with_company_detection(
    pine,
    query = "What is the vision and mission of farm fresh?",
    top_k = 8
)

2025-08-27 12:10:16,796 - retrieval - INFO - RAG start | query='What is the vision and mission of farm fresh?' | top_k=8
2025-08-27 12:10:17,351 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:10:25,862 - retrieval - INFO - Detected company: 'FARM_FRESH_BERHAD'
2025-08-27 12:10:25,863 - retrieval - INFO - Search query normalized: 'What is the vision and mission of farm fresh?' → 'What is the vision and mission of the company?'
2025-08-27 12:10:25,864 - retrieval - INFO - Query filter: {'from_company': 'FARM_FRESH_BERHAD'}
2025-08-27 12:10:35,477 - retrieval - INFO - Final answer: 

- Vision
  - "TO BE A SUSTAINABLE AND HONEST FOOD COMPANY THAT HAS AN INGRAINED CULTURE OF PLACING THE WELL-BEING OF CONSUMERS FIRST, CULMINATING IN STRONG BRAND LOVE." 

- Mission
  - "Diversifying our product portfolio to capture a wide, addressab